# Proyecto #3: Clustering - Inteligencia artificial

## Integrantes

- Pedro Domínguez
- Luis Chahua
- Carlos Guerrero

## Librerías

In [44]:
import pandas as pd
import numpy as np
import os

#Librerías que permitirá reducir dimensiones
import pywt
import pywt.data

#Librerías para lectura/edición de imágenes
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import rgb2gray

#Librerías para generar gráficas
import matplotlib.pyplot as plt
from tabulate import tabulate

#Librerías de modelos de clasificación
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold # k fold cross validation
from sklearn import metrics # k fold cross validation
from sklearn.metrics import recall_score, average_precision_score, roc_auc_score, f1_score, precision_score

## Lectura de datos

In [86]:
dataset_txt = "../data/dataset_tissue.txt"

dataset = pd.read_csv(dataset_txt).transpose()
dataset = dataset.drop(index = "Unnamed: 0", axis = 0).to_numpy()
#dataset.shape

types_txt = "../data/clase.txt"
types = pd.read_csv(types_txt, index_col = 0).to_numpy()
#types.shape

In [90]:
#Showing the dataset and types dimension
dataset.shape, types.shape

((189, 22215), (189, 1))